In [1]:
import matplotlib.pyplot as plt

In [2]:
import sys
from os.path import dirname
sys.path.append(dirname('./synthcity/src/'))

In [9]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

In [4]:
from sklearn.model_selection import train_test_split

In [21]:
from sklearn import preprocessing 

In [5]:
import torch.optim as optim
from torch.utils.data import DataLoader

In [6]:
from synthcity.plugins import Plugins
from synthcity.benchmark import Benchmarks

[KeOps] Warning : 
    The default C++ compiler could not be found on your system.
    You need to either define the CXX environment variable or a symlink to the g++ command.
    For example if g++-8 is the command you can do
      import os
      os.environ['CXX'] = 'g++-8'
    
[KeOps] Warning : Cuda libraries were not detected on the system ; using cpu only mode


In [7]:
from synthcity.utils.datasets.time_series.pbc import PBCDataloader
from synthcity.utils.datasets.time_series.google_stocks import GoogleStocksDataloader
from synthcity.plugins.core.dataloader import TimeSeriesDataLoader

In [36]:
data = pd.read_csv("data\\card_transaction.v1.csv")

In [22]:
label_encoder = preprocessing.LabelEncoder()

In [12]:
data

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,NaN,No
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,NaN,No
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,NaN,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24386895,1999,1,2020,2,27,22:23,$-54.00,Chip Transaction,-5162038175624867091,Merrimack,NH,3054.0,5541,NaN,No
24386896,1999,1,2020,2,27,22:24,$54.00,Chip Transaction,-5162038175624867091,Merrimack,NH,3054.0,5541,NaN,No
24386897,1999,1,2020,2,28,07:43,$59.15,Chip Transaction,2500998799892805156,Merrimack,NH,3054.0,4121,NaN,No
24386898,1999,1,2020,2,28,20:10,$43.12,Chip Transaction,2500998799892805156,Merrimack,NH,3054.0,4121,NaN,No


In [37]:
data["Year"] = label_encoder.fit_transform(data["Year"])
data["Merchant Name"] = label_encoder.fit_transform(data["Merchant Name"])
data["Merchant City"] = label_encoder.fit_transform(data["Merchant City"])
data["Merchant State"] = label_encoder.fit_transform(data["Merchant State"])
data["Zip"] = label_encoder.fit_transform(data["Zip"])
data["Use Chip"] = label_encoder.fit_transform(data["Use Chip"])
data['Amount'] = data['Amount'].str[1:]
data['Amount'] = data['Amount'].astype("float")
data['Amount'] = data['Amount']/data['Amount'].abs().max()

In [46]:
data = data.drop('Errors?', axis=1)
data['Is Fraud?'] = label_encoder.fit_transform(data["Is Fraud?"])

In [47]:
data

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Is Fraud?
0,0,0,11,9,1,06:21,0.010822,2,69374,6275,28,24756,5300,0
1,0,0,11,9,1,06:42,0.003106,2,46284,7828,28,24758,5411,0
2,0,0,11,9,2,06:22,0.009712,2,46284,7828,28,24758,5411,0
3,0,0,11,9,2,17:45,0.010407,2,68751,7828,28,24758,5651,0
4,0,0,11,9,3,06:23,0.008451,2,81833,6275,28,24756,5912,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24386895,1999,1,29,2,27,22:23,-0.004358,0,22204,7556,134,621,5541,0
24386896,1999,1,29,2,27,22:24,0.004358,0,22204,7556,134,621,5541,0
24386897,1999,1,29,2,28,07:43,0.004774,0,63763,7556,134,621,4121,0
24386898,1999,1,29,2,28,20:10,0.003480,0,63763,7556,134,621,4121,0
